In [1]:


from GenZ import decode_moddeling, prefill_moddeling, get_configs

import pandas as pd
from plotnine import *
import plotnine as p9
import pandas as pd
from GenZ.analyse_model import *


In [2]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2   

In [3]:
D = get_configs('GPT-4').hidden_size
V = 50257
model_df, summary_table = decode_moddeling(model = 'gpt-4', batch_size = 1, Bb = 1 , bits='int8',
                            input_tokens = 1, output_tokens = 0, model_profilling=True,
                            tensor_parallel = 1, pipeline_parallel = 1, debug=True) 
display(model_df)
display(summary_table)
# print(f'Model Weights={summary_table['Total Weights (MB)'][0]*2**20/1e12} T')
# print(f'Attn Size:{(model_df.loc[0,'Input_w (MB)'] + model_df.loc[5,'Input_w (MB)'])*120*2**20/1e9} B')
# print(f'Each MLP Size:{sum(model_df.loc[6:,'Input_w (MB)'])*120*2**20/1e9/16} B')
# print(f'FWD pass MLP Size:{sum(model_df.loc[6:,'Input_w (MB)'])*120/1024/16} B')

/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_decode.py:70: UserWarning: All params would not fit on chip. System Memory Cap:40.0 GB , Weights : 1820.4298095703125 GB, KV Cache:0.0025634765625 


,Layer Name,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,Repeat,Repeat,128,0,0.0,0.0,0.0,0.0,0.0
1,QKV,GEMM,"[((1, 10752, 1), (32256, 10752), (1, 32256, 1))]",0.999876,346.816512,0.010254,330.75,0.030762,330.791016
2,Logit Pre,Logit,"((1, 84, 1, 128), (1, 84, 1, 128), (1, 84, 1, 1))",0.498054,0.010752,0.010254,0.010254,0.00008,0.020588
3,Logit Suf,Logit,"((1, 84, 1, 128), (1, 84, 0, 128), (1, 84, 1, 0))",0.0,0.0,0.010254,0.0,0.0,0.010254
4,Attend Pre,Attend,"((1, 84, 1, 1), (1, 84, 1, 128), (1, 84, 1, 128))",0.498054,0.010752,0.00008,0.010254,0.010254,0.020588
5,Attend Suf,Attend,"((1, 84, 1, 0), (1, 84, 0, 128), (1, 84, 1, 128))",0.0,0.0,0.0,0.0,0.010254,0.010254
6,Out Proj,GEMM,"[((1, 10752, 1), (10752, 10752), (1, 10752, 1))]",0.999814,115.605504,0.010254,110.25,0.010254,110.270508
7,Gate,GEMM,"[((1, 10752, 1), (16, 10752), (1, 16, 1))]",0.941094,0.172032,0.010254,0.164062,0.000015,0.174332
8,up+gate,GEMM,"[((1, 10752, 1), (86016, 10752), (1, 86016, 1))]",0.999895,924.844032,0.010254,882.0,0.082031,882.092285
9,down,GEMM,"[((1, 86016, 1), (10752, 86016), (1, 10752, 1))]",0.999895,924.844032,0.082031,882.0,0.010254,882.092285


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB)
0,297353.87136,1.864161e+06,1864120.125,1580544.0,2.625,6174.0


In [4]:
for model in ['gpt-4']:
    model_df, summary_table = prefill_moddeling(model = model, batch_size = 1,
                            input_tokens = 1024, model_profilling=True,
                            tensor_parallel = 8, pipeline_parallel = 16) 
    print(model)
    display(model_df)
    display(summary_table)


gpt-4


/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_prefill.py:24: UserWarning: Batch size is divided into micro batches for pipeline parallel, micro batch size:1, consider increasing batch size


,Layer Name,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,embeddings,GEMM,"[((1, 16032, 1024), (10752, 16032), (1, 10752,...",441.718081,176513.089536,31.3125,328.78125,21.0,381.09375
1,Emb_AR,Sync,"(1, 1024, 10752)",0,0.0,0.0,0.0,0.0,0.0
2,Repeat,Repeat,15,0,0.0,0.0,0.0,0.0,0.0
3,Repeat,Repeat,8,0,0.0,0.0,0.0,0.0,0.0
4,QKV,GEMM,"[((1, 10752, 1024), (4224, 10752), (1, 4224, 1...",382.757282,46506.442752,21.0,86.625,8.25,115.875
5,Logit,Logit,"((1, 11, 1024, 128), (1, 11, 1024, 128), (1, 1...",51.2,1476.395008,2.75,2.75,22.0,27.5
6,Attend,Attend,"((1, 11, 1024, 1024), (1, 11, 1024, 128), (1, ...",51.2,1476.395008,22.0,2.75,2.75,27.5
7,Out Proj,GEMM,"[((1, 1408, 1024), (10752, 1408), (1, 10752, 1...",280.931116,15502.147584,2.75,28.875,21.0,52.625
8,MHA AR,Sync,"(1, 1024, 10752)",0,0.0,0.0,0.0,0.0,0.0
9,Gate,GEMM,"[((1, 1344, 1024), (16, 1344), (1, 16, 1024))]",7.785663,22.020096,2.625,0.041016,0.03125,2.697266


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB)
0,3.897619e+07,487347.4375,467030.8125,0.0,704.0,1787.625


In [5]:
prefill_moddeling(model = 'gpt-4', batch_size = 1,
                                            input_tokens = 2000,
                                            system_name = 'H100_GPU',
                                            bits='int8',
                                            tensor_parallel = 8, pipeline_parallel = 16, debug=False) 

/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_prefill.py:24: UserWarning: Batch size is divided into micro batches for pipeline parallel, micro batch size:1, consider increasing batch size


{'Latency': 118.45501024974416,
 'Throughput': 8.442023666974102,
 'Runtime_breakdown': Embedding: 1.2915754282245075, MHA: 31.325471460165993, FFN: 85.0773913398693, Collective: 33.97996026441786, LA_layers: 1.4578281092012135, QKVO_layers: 15.674567830131444, softmax: 0, AR_time: 33.21938824293349, A2A_time: 0, Send_Recv_time: 0.760572021484375,
 'is_offload': False}

In [6]:
decode_moddeling(model = 'gpt-4', batch_size = 1, Bb = 4 ,
                                            input_tokens = 2000, output_tokens = 256, 
                                            system_name = 'H100_GPU',
                                            bits='int8',
                                            tensor_parallel = 8, pipeline_parallel = 16, debug=False) 

/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_decode.py:27: UserWarning: Batch size is divided into micro batches for pipeline parallel, micro batch size:1, consider increasing batch size


{'Latency': 23.40510223723393,
 'Throughput': 42.72572663276614,
 'Runtime_breakdown': Embedding: 0.06717536064222747, MHA: 6.666922243105821, FFN: 16.576669489442914, Collective: 12.826170153723826, LA_layers: 0.2991034688513263, QKVO_layers: 2.1238806232128264, softmax: 0, AR_time: 12.731835009680857, A2A_time: 0, Send_Recv_time: 0.09433514404296875,
 'is_offload': False}